# Datasets
A brainstorm of what datasets would be useful

## By player per game
A dictionary of player's and their stats in all their games

## By team per game
A dictionary of all teams and overall stats for each game they've played

## Ratings
A dictionary of all players and their current ratings. Not on a game by game basis, but an aggregate.
Sources could include:
* NBA official stats
* 2K
* Other free stats sources

## Data structure
It might be useful to have custom data structures to access all this data. For example, I could have a specific data structure for box score, player stats, team stats.

# Using tutorial
[Here](http://practicallypredictable.com/2017/12/21/web-scraping-nba-team-matchups-box-scores/) is a nice tutorial on how to scrape box score data.
A nice notebook viewer version [here](https://nbviewer.jupyter.org/github/practicallypredictable/posts/blob/master/basketball/nba/notebooks/scrape-stats_nba-team_matchups.ipynb)

In [1]:
from itertools import chain
from pathlib import Path
from time import sleep
import datetime as dt
import requests
from tqdm import tqdm
tqdm.monitor_interval = 0
import numpy as np
import pandas as pd
from collections import namedtuple
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### User agent
Found out what my user-agent is [here](https://www.whoishostingthis.com/tools/user-agent/)

In [2]:
USER_AGENT = ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' +
              'AppleWebKit/537.36 (KHTML, like Gecko) ' +
              'Chrome/70.0.3538.77 Safari/537.36')

REQUEST_HEADERS = {
    'user-agent': USER_AGENT,
}

### Getting gameid from game by game page

Make a function to return all GameIDs in a specified time period.

In [3]:
NBA_ID = '00'

NBA_SEASON_TYPES = {
    'regular': 'Regular Season',
    'playoffs': 'Playoffs',
    'preseason': 'Pre Season',
}

season_type = NBA_SEASON_TYPES['regular']


In [66]:
def get_game_info(date):
    '''
    Given a date, find all the GameIDs for that day. 
    Takes in a datetime.date input and returns a named tuple consisting of GameID, Matchup, and date
    '''

    NBA_URL = 'https://stats.nba.com/stats/teamgamelogs'

    date_from = date
    date_to = date_from

    date_from_string = date_from.strftime('%m/%d/%Y')
    date_to_string = date_from_string

    # Determine which NBA season
    if date_from.month >= 10:
        season_start_year = date_from.year
    else:
        season_start_year = date_from.year - 1
        
    season = str(season_start_year) + '-' + str(season_start_year + 1)[-2:]

    nba_params = {
        'LeagueID': NBA_ID,
        'Season': season,
        'SeasonType': season_type,
        'DateFrom': date_from_string, 
        'DateTo': date_to_string,
        'Outcome': '' # To prevent doubling up and recording both the winner and loser of one game, put 'W' or 'L'
    }

    r = requests.get(NBA_URL, params=nba_params, headers=REQUEST_HEADERS, allow_redirects=False, timeout=15)
    assert r.status_code == 200
    
    json_dict = r.json() # Turns the json text into a python dict
    games_dict = json_dict['resultSets'][0] # Only has one element
    headers = games_dict['headers']
    
    GAME_ID_COL = headers.index('GAME_ID')
    DATE_COL = headers.index('GAME_DATE')
    MATCHUP_COL = headers.index('MATCHUP')
    games_list = games_dict['rowSet'] # List of list
    
    df_games = pd.DataFrame(columns=headers, data=games_list)
    df_games = df_games[['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION',
                         'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL',
                         'PTS']]
    df_games['HOME_AWAY'] = df_games['MATCHUP'].apply(lambda x: 'HOME' if  'vs.' in x else 'AWAY')
    
    # Get the opponent's info
    df_games['TEAM_ABBREVIATION_OPP'] = df_games['MATCHUP'].apply(lambda x: x[-3:])
    df_games = pd.merge(df_games, df_games[['TEAM_ID', 'TEAM_ABBREVIATION', 'PTS']], 
                        left_on='TEAM_ABBREVIATION_OPP', right_on = 'TEAM_ABBREVIATION',
                        suffixes=('', '_OPP'))
    df_games = df_games.rename(columns={'TEAM_ABBREVIATION_OPP': 'OPP_TEAM_ABBREVIATION',
                             'PTS_OPP': 'OPP_PTS',
                             'TEAM_ID_OPP': 'OPP_TEAM_ID'
                            })
    
    # Remove duplicate column of 'OPP_TEAM_ABBREVIATION'
    df_games = df_games.loc[:,~df_games.columns.duplicated()]

    return df_games

In [67]:
get_game_info(dt.date(2018,11,7))

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_PTS
0,2018-19,1610612742,DAL,0021800160,2018-11-07T00:00:00,DAL @ UTA,L,102,AWAY,UTA,1610612762,117
1,2018-19,1610612739,CLE,0021800153,2018-11-07T00:00:00,CLE vs. OKC,L,86,HOME,OKC,1610612760,95
2,2018-19,1610612741,CHI,0021800159,2018-11-07T00:00:00,CHI @ NOP,L,98,AWAY,NOP,1610612740,107
3,2018-19,1610612758,SAC,0021800161,2018-11-07T00:00:00,SAC vs. TOR,L,105,HOME,TOR,1610612761,114
4,2018-19,1610612759,SAS,0021800156,2018-11-07T00:00:00,SAS @ MIA,L,88,AWAY,MIA,1610612748,95
5,2018-19,1610612753,ORL,0021800154,2018-11-07T00:00:00,ORL vs. DET,L,96,HOME,DET,1610612765,103
6,2018-19,1610612754,IND,0021800157,2018-11-07T00:00:00,IND vs. PHI,L,94,HOME,PHI,1610612755,100
7,2018-19,1610612737,ATL,0021800155,2018-11-07T00:00:00,ATL vs. NYK,L,107,HOME,NYK,1610612752,112
8,2018-19,1610612750,MIN,0021800162,2018-11-07T00:00:00,MIN @ LAL,L,110,AWAY,LAL,1610612747,114
9,2018-19,1610612743,DEN,0021800158,2018-11-07T00:00:00,DEN @ MEM,L,87,AWAY,MEM,1610612763,89


### Parsing the box score
Now that we have a way to get the game ids, we have to parse the box score

The XHR with individual player scores is boxscoretraditionalv2

example of url is 

https://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021800162&RangeType=0&Season=2018-19&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

Not sure what a bunch of these parameters are, going to compare them to others. This is an example from a different game, also final boxscore:

https://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=28800&GameID=0021800152&RangeType=0&Season=2018-19&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

It has the exact same parameters except for the GameID, I suspect that the other parameters change when the game is not final. At time of writing, there are no ongoing games so I shall try again tomorrow.

When a game is in play, there is no boxscoretraditionalv2 being called. Replacing the above exmple urls with a live game's GameID shows null for everyone's stats. Ways to handle this:
* Find the real time boxscores and update as we go
* Prevent any update for non finished games
    * Check NULL for every boxscore value
    * Check something else to see if a game is live. This is preferable.
    
    

In [69]:
def parse_boxscore(GameID, season, season_type=NBA_SEASON_TYPES['regular']):
    ''' 
    Takes in the GameID in string format and returns a dataframe of the boxscore, broken down by player.
    If game is not yet finished, will return a boxscore with NULL values
    >>> df_boxscore = parse_boxscore('0021800151')
    '''
    if not isinstance(GameID, str):
        raise TypeError('GameID must be string')
    
    URL_GAME_BOXSCORE = 'https://stats.nba.com/stats/boxscoretraditionalv2'
    boxscore_params = {
        'GameID': GameID,
        'Season': season,
        'SeasonType': season_type,
        'EndPeriod': '10',
        'EndRange': '28800',
        'RangeType': '0',
        'StartPeriod': '1',
        'StartRange': '0'
    }

    r_boxscore = requests.get(URL_GAME_BOXSCORE, params=boxscore_params, 
                              headers=REQUEST_HEADERS, allow_redirects=False, timeout=15)

    assert r_boxscore.status_code == 200

    json_boxscore = r_boxscore.json()

    player_stats = json_boxscore['resultSets'][0] # 0 is player stats, 1 is team stats, 2 is starter bench stats
    boxscore_headers = player_stats['headers'] 
    boxscore_stats = player_stats['rowSet']

    df_boxscore = pd.DataFrame(columns=boxscore_headers, data=boxscore_stats)
    return df_boxscore

In [70]:
parse_boxscore('0021800151', '2018-19')

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0021800151,1610612751,BKN,Brooklyn,203925,Joe Harris,F,,28:19,5.0,11.0,0.455,1.0,6.0,0.167,0.0,0.0,0.000,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,11.0,23.0
1,0021800151,1610612751,BKN,Brooklyn,201162,Jared Dudley,F,,24:29,2.0,7.0,0.286,1.0,5.0,0.200,0.0,0.0,0.000,0.0,2.0,2.0,1.0,0.0,0.0,2.0,4.0,5.0,5.0
2,0021800151,1610612751,BKN,Brooklyn,1628386,Jarrett Allen,C,,24:20,4.0,7.0,0.571,0.0,1.0,0.000,2.0,2.0,1.000,2.0,7.0,9.0,5.0,1.0,1.0,1.0,2.0,10.0,13.0
3,0021800151,1610612751,BKN,Brooklyn,1627747,Caris LeVert,G,,32:12,10.0,16.0,0.625,3.0,6.0,0.500,3.0,4.0,0.750,1.0,4.0,5.0,1.0,1.0,1.0,3.0,2.0,26.0,9.0
4,0021800151,1610612751,BKN,Brooklyn,1626156,D'Angelo Russell,G,,26:55,6.0,15.0,0.400,1.0,6.0,0.167,2.0,2.0,1.000,1.0,5.0,6.0,3.0,2.0,0.0,0.0,3.0,15.0,7.0
5,0021800151,1610612751,BKN,Brooklyn,203459,Allen Crabbe,,,22:25,3.0,12.0,0.250,1.0,5.0,0.200,0.0,0.0,0.000,3.0,4.0,7.0,2.0,0.0,1.0,1.0,2.0,7.0,12.0
6,0021800151,1610612751,BKN,Brooklyn,202334,Ed Davis,,,23:40,4.0,7.0,0.571,0.0,0.0,0.000,1.0,5.0,0.200,4.0,8.0,12.0,0.0,1.0,0.0,0.0,3.0,9.0,9.0
7,0021800151,1610612751,BKN,Brooklyn,1626178,Rondae Hollis-Jefferson,,,12:33,1.0,6.0,0.167,0.0,1.0,0.000,3.0,4.0,0.750,0.0,2.0,2.0,1.0,1.0,0.0,2.0,0.0,5.0,-1.0
8,0021800151,1610612751,BKN,Brooklyn,203915,Spencer Dinwiddie,,,29:56,5.0,7.0,0.714,2.0,4.0,0.500,0.0,0.0,0.000,0.0,5.0,5.0,3.0,2.0,1.0,0.0,4.0,12.0,20.0
9,0021800151,1610612751,BKN,Brooklyn,203894,Shabazz Napier,,,15:12,1.0,4.0,0.250,0.0,2.0,0.000,2.0,2.0,1.000,2.0,2.0,4.0,5.0,1.0,0.0,1.0,0.0,4.0,13.0
